# Activity 2:
## Gaussian Eliminiation and LU Factorizations

In [ ]:
import numpy as np
from numpy import random as RA

In this activity, we will work to build an LU factorization algorithm from the ground up using only the most basic of built-in numpy commands. First, we'll review some array manipulation.

In [ ]:
A=np.array([[1,2,4,-3],[2,6,1,7],[-0.5,9,3,3],[-1,2,-4,3]])

You can change the elements of an array individually like so:

In [ ]:
B=A.copy()
print("B=\n",B)
B[0][0]=-99
B[2][3]=99
print("B=\n",B)

(recall the python indexing conventions: the top row/leftmost column are row/column zero)

Arithmetic operations performed on arrays (or rows of arrays) are performed element-wise. For example:

In [ ]:
B=A.copy()
B[3]=4*A[2] #change the fourth row of B to be four times the third row of A
print("A=\n", A,"\nB=\n",B) #display A and B readably
C=np.identity(4)-3*A #construct a new matrix given as the difference between the identity matrix and 3 times A
print("C=\n",C) #display C readably


You can also reference rows or elements of the same array.

In [ ]:
B=A.copy()
print("B=\n",B)
B[0]=3*B[1]
print("B=\n",B)
B[1]=B[0][3]*B[1]
print("B=\n",B)

You can also swap rows of an array by calling a list of indices.

In [ ]:
B=A.copy()
print("B=\n",B)
B[[0,3]]=B[[3,0]] #read this as "set the 0th row and the 3rd row of B to be the 3rd and 0th rows of B"
print("B=\n",B)
B[[1,2,3]]=B[[3,1,2]]
print("B=\n",B)

You try: copy $A$ (from above) to a new array $B$. move the bottom row to the top and shift the other rows down. Then, negate the new bottom row and multiply the new top row by $2$. Then, let $C=B-A$.

In [ ]:
B=A.copy()
B[[0,1,2,3]]=B[[3,0,1,2]]
B[3]=-B[3]
B[0]=2*B[0]

C=B-A

print("A=\n",A,"\nB=\n",B,"\nC=\n",C)
#desired output:
#A=
# [[ 1.   2.   4.  -3. ]
# [ 2.   6.   1.   7. ]
# [-0.5  9.   3.   3. ]
# [-1.   2.  -4.   3. ]] 
#B=
# [[-2.   4.  -8.   6. ]
# [ 1.   2.   4.  -3. ]
# [ 2.   6.   1.   7. ]
# [ 0.5 -9.  -3.  -3. ]] 
#C=
# [[ -3.    2.  -12.    9. ]
# [ -1.   -4.    3.  -10. ]
# [  2.5  -3.   -2.    4. ]
# [  1.5 -11.    1.   -6. ]]


## Exercise 1: Regular Gaussian Elimination
First, let's build an function to perform Gaussian Elimination in the case of a regular matrix.
You are free to build your algorithm as you see fit (without using built-in functions which do the hard part for you), but here is a basic outline of one:
* Set a variable n for the size of `A` using the `shape` method. `A.shape` returns a tuple with the dimensions of `A`, so if `A` is $n\times n$, `A.shape[0]` will return `n`.
* Copy A to a new array `U` (we'll generally avoid altering the input of our functions when unnecessary)
* Use a for loop to iterate over `i` in `range(n)`. This parameter will represent the pivot we are currently working with. Then, within the for loop:
    * "Clear out" column i below the pivot `U[i][i]` by iterating with a new for loop over `j` in `range(i+1,n)`. This `j` will parameterize the row we are currently "clearing out."
        * Change the `j`th row `U[j]` by subtracting the appropriate multiple of the `i`th row `U[i]` to clear out the $(i,j)$th entry `U[i][j]`.

For the moment, we'll assume that the input is regular--don't worry for now about what happens in the case it is not.

In [ ]:
def my_elim(A): #The input will be a regular matrix A, encoded as a np.array
    n=A.shape[0] #get number of rows
    U=A.copy() #initialize U
    for i in range(n): #parameterize pivots
        for j in range(i+1,n): #parameterize entries below the pivot U[i][i]
            m = U[j][i] / U[i][i] #compute which multiple of U[i] we'll need to subtract
            U[j] = U[j] - m * U[i] #perform the subtraction            
    return U #The output will be an upper-triangular matrix, also encoded as a np.array

Try out your elimination function on the test matrix below (you have our word that it's regular).

In [ ]:
A=np.array([[5., 4., 0.],
        [6., 5., 4.],
        [0., 6., 5.]])
my_elim(A)
#desired output: 
#        array([[   5. ,    4. ,    0. ],
#               [   0. ,    0.2,    4. ],
#               [   0. ,    0. , -115. ]])

## Exercise 2: LU Factorization
Now, we'll add in code to compute the LU factorization of `A`.
First, we'll need code to construct elementary matrices.
In the cell below, define a function to give the $n\times n$ elementary matrix $E$ where $E_{ij}=a$.

One helpful numpy built-in function is `np.identity`. Try it out in the next cell

In [ ]:
np.identity(5)

**Exercise: (a)** Now, use that to write your `elem_matrix` function.

In [ ]:
def elem_matrix(n,i,j,a):
    E=np.identity(n)
    E[i][j]=a
    return E

**(b)** Now let's modify your `my_elim` function to produce an LU factorization of your input matrix `A`.

Some things to keep in mind:
* You still don't need to worry about the case that `A` is irregular--for now, assume it is regular. 
* The `np.dot` function performs matrix multiplication (that is, $AB$ is given by `np.dot(A,B)`).
* Page 17 of Olver-Shakiban gives the formula you'll need to form `L`. Initialize it as the appropriately-sized identity matrix, and multiply by the appropriate elementary matrix in each step of the iteration

In [ ]:
def my_LU(A): #The input will be a regular matrix A, encoded as a np.array
    n=A.shape[0] #get number of rows
    U=A.copy() #initialize U
    L=np.identity(n) #initialize L
    for i in range(n): #parameterize pivots
        for j in range(i+1,n): #parameterize entries below the pivot U[i][i]
            m=U[j][i]/U[i][i] #compute which multiple of U[i] we'll need to subtract
            U[j]=U[j] - m * U[i] #perform the subtraction
            L=np.dot(L , elem_matrix(n,j,i,m) ) #multiply L on the right by the elementary matrix representing the inverse of our row operation
    return (L,U)

Try it out in the next two cells.

In [ ]:
A=np.array([[5., 4., 0.],
        [6., 5., 4.],
        [0., 6., 5.]])
(L,U)=my_LU(A)
print("L=\n",L,"\nU=\n",U)
#desired output:
#    (array([[ 1. ,  0. ,  0. ],
#            [ 1.2,  1. ,  0. ],
#            [ 0. , 30. ,  1. ]]),
#     array([[   5. ,    4. ,    0. ],
#            [   0. ,    0.2,    4. ],
#            [   0. ,    0. , -115. ]]))

In [66]:
#Check if reconstruction works!
print("A=\n",A,"\nLU=\n",np.dot(L,U))

A=
 [[0.14509204 0.32108452 0.39526872 0.86278664 0.31933496 0.40132048
  0.30546698 0.11704652 0.25564367 0.17806998]
 [0.35490346 0.3590646  0.83303829 0.08531373 0.3578937  0.8892153
  0.99775313 0.26251429 0.79679859 0.02588547]
 [0.89718993 0.75091897 0.43887796 0.35304412 0.20679536 0.62063301
  0.76639068 0.8813102  0.13003238 0.0479438 ]
 [0.28953305 0.14521926 0.86102509 0.18314289 0.39786691 0.21598413
  0.62658133 0.72176809 0.59846813 0.18931132]
 [0.76600162 0.06075262 0.35235814 0.85963326 0.66441031 0.57166466
  0.19309368 0.65639484 0.57665367 0.93116179]
 [0.06269821 0.88204077 0.31799626 0.50472747 0.35745125 0.91730262
  0.6717936  0.01234591 0.64430528 0.34120225]
 [0.03858558 0.13535752 0.70106743 0.66115775 0.10221863 0.86961372
  0.03702521 0.31006449 0.52700891 0.38440762]
 [0.19453926 0.00799612 0.69836337 0.93137464 0.12885283 0.6763548
  0.90225647 0.36620062 0.35175356 0.55851439]
 [0.44277871 0.74976365 0.6536152  0.78599172 0.76741515 0.61592592
  0.562570

**(c)** So far, we haven't had to worry about the case of irregular input. However, in practice, your code needs to at least let the user know when they have entered as input an irregular matrix.
Python performs error handling by using the built-in `raise` command to raise an `Exception`, which is a message given to the user to inform them what's gone wrong. Below is a quick demonstration:

In [ ]:
def even_division(n):
    if n%2 == 1: #check if n odd
        raise Exception("input integer is odd")
    return n//2 #divide by 2
    

In [ ]:
even_division(4)

In [ ]:
#even_division(5)

**Exercise:**
In the below cell, modify your code for `my_LU` to raise an exception when the input is an irregular matrix. The `Exception` should say "Input matrix irregular."

In [ ]:
def my_LU_safe(A):
    n=A.shape[0] #get number of rows
    U=A.copy() #initialize U
    L=np.identity(n) #initialize L
    for i in range(n): #parameterize pivots
        if U[i][i]==0:
            raise Exception("Input matrix irrecular")
        for j in range(i+1,n): #parameterize entries below the pivot U[i][i]
            m=U[j][i]/U[i][i] #compute which multiple of U[i] we'll need to subtract
            U[j]=U[j] - m * U[i] #perform the subtraction
            L=np.dot(L , elem_matrix(n,j,i,m) ) #multiply L on the right by the elementary matrix representing the inverse of our row operation
    return (L,U)

In [ ]:
#A=np.array([[i*j for j in range(5)] for i in range(5)]) #This should throw an exception
#my_LU_safe(A)

### Exercise 3: Solving Linear Systems
One of the major use cases for LU factorization is solving systems by forward- and back- substitution, as described in Olver-Shakiban, section 1.3. 
To solve a system $A\vec{x}=\vec{b}$, this works by
* computing an LU factorization $A=LU$, 
* Solving the system $L\vec{c}=\vec{b}$
* Solving the system $U \vec{x}= \vec{c}$

**Exercise:** Below, give code which implements this using `my_LU`. Do *not* use any `numpy` built-in functions which trivialize any part of this (such as taking matrix inverses)--rather you should write code to perform the substitution manually. You may choose to proceed by writing a function for back substitution then another for forward substitution if you wish--that approach is outlined below.
Be sure to test your code before you submit!

In [ ]:
def forward_sub(L,b): #input: lower-triangular matrix L, vector of compatible dimension b. 
    n=b.shape[0]
    c=np.zeros(n) #initialize output c
    for i in range(n):
        c[i]=(b[i]-sum([L[i][j]*c[j] for j in range(i)]))/L[i][i] #forward-substitution formula
    return c
def back_sub(U,c): #input: upper-triangular matrix L, vector of compatible dimension b. 
    n=c.shape[0]
    x=np.zeros(n) #initialize output x
    for i in range(n):
        x[-(1+i)]=(c[-(1+i)]-sum([U[-(i+1)][-(j+1)]*x[-(j+1)] for j in range(i)]))/U[-(i+1)][-(i+1)] #back-substitution formula.
    return x 

In [ ]:
def my_solver(A,b):
    (L,U)=my_LU(A)
    c=forward_sub(L,b)
    x=back_sub(U,c)
    return x

In [ ]:
#testing your code
A=RA.rand(10,10)
b=RA.rand(10)
x=my_solver(A,b)
Ax=np.dot(A,x)
print(Ax-b) #should get vector of zeros, or close to it.
max(Ax-b)<10**(-13) #check that result is within numerical error of correct

### Exercise 4 (Challenge!): Hilbert Matrices and Numerical Instability.
LU sadly suffers from some numerical instability. 
This rears its head in particular when the input matrix $A$ is *ill-conditioned*, meaning that the product $A\vec x$ can change significantly with relatively small perturbations of the input. 
A classic example is the $n$th *Hilbert matrix* $H_n$, given entrywise by $$H_n=\left(\frac{1}{i+j-1}\right)_{i,j}$$ where $i,j$ range over $1,\dots,n$.
For example, the third Hilbert matrix is 
$$H_3=\begin{pmatrix}
1 & \frac12 & \frac13\\
\frac12 & \frac13 & \frac14\\
\frac13 & \frac14 & \frac15
\end{pmatrix}.$$

**Exercise: (a)** Write a function `hilbert` which takes as input an integer `n` and returns the `n`th Hilbert matrix $H_n$. Be careful to account for python's indexing.

In [ ]:
def hilbert(n):
    return np.array([[1/(i+j+1) for i in range(n)] for j in range(n)])

In [ ]:
hilbert(5)

**(b):** A good way to check the numerical accuracy of a linear system solver such as `my_solver` for a given matrix $A$ is to generate a random vector $\vec x_0$ of appropriate dimension, set $\vec b$ to be the product $\vec b = A \vec x_0$, and then set $\vec x_1$ to be the result of running the solver on the linear system $A\vec x= \vec b$.
In principle, $\vec x_1$ should be equal to $\vec x_0$, but as we'll see, thanks to rounding errors, this is not always the case in practice. 
Write a function `hilbert_checker` takes as input a parameter `max_discrepancy` and then loops to perform the process above for increasing dimensions $n$ starting with $n=1$. Use `np.random.rand` (imported for convenience as `RA.rand`) to generate your random vectors. At each stage in the loop, print the current entry `n` and the maximum entry of the absolute difference $\left \lvert \vec x_0- \vec x_1\right \rvert$. Your loop should terminate when the maximum difference in absolute value between $\vec x_0$ and $\vec x_1$ exceeds `max_discrepancy`, returning the dimension `n` at which that occurred. How long does it take for `max_discrepancy` to exceed $10^{-8}$? What about $10^{-4}$,  $10^{-2}$,  1, 100, or 10000? Try running `hilbert_checker` a few times for the same value of `max_discrepancy`. Do you get similar results? Write a few words below summarizing your observations.

*Note:* While the python built-in function `abs` does work as expected with `numpy` arrays, the same cannot be said for `max`. Rather, use `np.max`.

In [ ]:
def hilbert_checker(max_discrepancy=1):
    n=0
    discrepancy=0
    while discrepancy < max_discrepancy:
        n+=1
        x_0=RA.rand(n)
        H_n=hilbert(n)
        b=np.dot(H_n,x_0)
        x_1=my_solver(H_n,b)
        discrepancy=np.max(abs(x_1-x_0))
        print("n=",n," discrepancy=",discrepancy)
    return n
#for convenience, a version without printing each time
def hilbert_checker_quiet(max_discrepancy=1):
    n=0
    discrepancy=0
    while discrepancy < max_discrepancy:
        n+=1
        x_0=RA.rand(n)
        H_n=hilbert(n)
        b=np.dot(H_n,x_0)
        x_1=my_solver(H_n,b)
        discrepancy=np.max(abs(x_1-x_0))
        #print("n=",n," discrepancy=",discrepancy)
    return n

In [ ]:
hilbert_checker(10**-8)

In [ ]:
hilbert_checker(10**-4)

In [ ]:
hilbert_checker(10**-2)

In [ ]:
hilbert_checker(10**0)

In [ ]:
hilbert_checker(100)

In [ ]:
hilbert_checker_quiet(10**4)